* 訓練データセットをロード

In [ ]:
from sklearn.datasets import load_files

In [9]:
reviews_train = load_files("data/aclImdb/train/")

In [13]:
# load_filesは一連の訓練テキストと訓練ラベルを返す
text_train, y_train = reviews_train.data, reviews_train.target

In [14]:
print("type of text_train: {}".format(type(text_train)))
print("length of text_train: {}".format(len(text_train)))
print("text_train[1]:\n{}".format(text_train[1]))

type of text_train: <class 'list'>
length of text_train: 75000
text_train[1]:
b"Amount of disappointment I am getting these days seeing movies like Partner, Jhoom Barabar and now, Heyy Babyy is gonna end my habit of seeing first day shows.<br /><br />The movie is an utter disappointment because it had the potential to become a laugh riot only if the d\xc3\xa9butant director, Sajid Khan hadn't tried too many things. Only saving grace in the movie were the last thirty minutes, which were seriously funny elsewhere the movie fails miserably. First half was desperately been tried to look funny but wasn't. Next 45 minutes were emotional and looked totally artificial and illogical.<br /><br />OK, when you are out for a movie like this you don't expect much logic but all the flaws tend to appear when you don't enjoy the movie and thats the case with Heyy Babyy. Acting is good but thats not enough to keep one interested.<br /><br />For the positives, you can take hot actresses, last 30 minutes,

* HTMLの改行シーケンス(<br /\>)を取り除く

In [22]:
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

In [23]:
text_train[0]

b'Full of (then) unknown actors TSF is a great big cuddly romp of a film.  The idea of a bunch of bored teenagers ripping off the local sink factory is odd enough, but add in the black humour that Forsyth & Co are so good at and your in for a real treat.  The comatose van driver by itself worth seeing, and the canal side chase is just too real to be anything but funny.  And for anyone who lived in Glasgow it\'s a great "Oh I know where that is" film.'

In [25]:
import numpy as np

In [26]:
print("Samples per class (training): {}".format(np.bincount(y_train)))

Samples per class (training): [12500 12500 50000]


* テストデータセットもロードする

In [27]:
reviews_test = load_files("data/aclImdb/test/")

In [28]:
text_test, y_test = reviews_test.data, reviews_test.target

In [30]:
print("Number of documents in test data: {}".format(len(text_test)))
print("Samples per class (test): {}".format(np.bincount(y_test)))

Number of documents in test data: 25000
Samples per class (test): [12500 12500]


In [31]:
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

* あるレビューに対して、レビューのテキストに基づいて「肯定的」もそくは「否定的」のラベルを割り当てる。これは2クラス分類タスク
* しかし、テキストデータは機械学習が扱えるような形式になっていない。
* テキストの文字列表現を機械学習のアルゴリズムを適用できるような数値表現に変換する必要がある

# Bag of Wordsによるテキスト表現

* 最も単純で効率がよく、機械学習で広く用いられているテキストデータ表現がBoW(bag-of-words)表現である。
* この表現では、章立て、パラグラフ、文章、フォーマットなどの入力テキストの持つ構造のほとんどすべてが失われる
* コーパスに現れた単語が**テキストに現れる回数**だけが数えられる
* 構造を捨て単語の現れる回数だけを数えるので、テキストを単語を入れる「袋(bag)」と考える

文書の集合であるコーパスに対してBoW表現を計算するには次の3ステップが必要になる。

1. トークン分割：個々の文書を単語（トークン）に分割する
2. ボキャブラリ構築：すべての文書に現れるすべての単語をボキャブラリとして集め、番号を付ける（例えばアルファベット順で）
3. エンコード：個々の文書に対してボキャブラリの単語が現れる回数を数える。

## 7.3.1 トイデータセットに対するBoW
BoW表現は、変換器としてCountVectorizerに実装されている。まず2つのサンプルデータからなるトイデータセットで試してみよう。

In [32]:
bards_words = ["The fool doth think he is wise,",
               "but the wise man knows himself to be a fool"]

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
vect = CountVectorizer()

In [35]:
vect.fit(bards_words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

* CountVectorizerのfitでは、訓練データのトークン分割とボキャブラリの構築が行われる

In [36]:
print("Vocaburary size: {}".format(len(vect.vocabulary_)))
print("Vocaburary content:\n {}".format(vect.vocabulary_))

Vocaburary size: 13
Vocaburary content:
 {'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


* ボキャブラリには、"be"から"wise"までの13個の単語が含まれている。
* 訓練データに対するBoW表現を作るには、transformメソッドを呼び出す。

In [37]:
bag_of_words = vect.transform(bards_words)
print("bag_of_words: {}".format(repr(bag_of_words)))

bag_of_words: <2x13 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


In [38]:
print("Dense representation of bag_of_words:\n{}".format(
    bag_of_words.toarray()))

Dense representation of bag_of_words:
[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


1つ目の文は、'be'が0、'but'が0、'doth'が1。。。と表現されている